In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage


In [6]:
import sys
sys.path.append('../')
from Secrets.openai_key import google_api_key
import os
os.environ['GOOGLE_API_KEY'] = google_api_key

In [9]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Simple text invocation
result = llm.invoke("What is AMEX LUMI ?")
print(result.content)


E0000 00:00:1760217322.701933 4110982 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


**AMEX LUMI** (often just called **LUMI**) is a digital platform developed by American Express specifically designed to help **businesses that accept American Express cards** streamline and manage their B2B (business-to-business) payment processes.

In essence, it's a tool for businesses to better understand, reconcile, and process the payments they receive from other businesses using American Express cards.

Here's a breakdown of what it is and what it does:

1.  **Purpose:** To simplify and automate the reconciliation of American Express card payments received by businesses. B2B payments can be complex, involving detailed invoices, varying terms, and different payment methods. LUMI aims to bring clarity and efficiency to the Amex portion of this.

2.  **Target Audience:** Businesses that process a significant volume of B2B payments where their customers pay using American Express corporate or business cards. It's not for individual consumers managing their personal Amex cards.

3.  *

In [10]:
from langchain.document_loaders.csv_loader import CSVLoader


In [14]:
loader = CSVLoader(file_path='../data/data_faqs.csv',source_column='prompt')
data = loader.load()

In [40]:
# data

### Create the embadding data using Hugging face

In [32]:
from langchain_community.embeddings import HuggingFaceInstructEmbeddings


# model_name = "hkunlp/instructor-large"
# model_kwargs = {'device': 'cpu'}
# encode_kwargs = {'normalize_embeddings': True}
# embeddings = HuggingFaceInstructEmbeddings(
#     model_name=model_name,
#     model_kwargs=model_kwargs,
#     encode_kwargs=encode_kwargs
# )

In [33]:
# embeddings = HuggingFaceInstructEmbeddings(query_instruction="Represent the query for retrieval: ")
# e = embeddings.embed_query("How can I reduce my stress levels?")

/var/folders/ls/6h78t28x6c35lwkn1jh_nxc80000gn/T/ipykernel_30205/2249563508.py:1: LangChainDeprecationWarning: Default values for HuggingFaceInstructEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceInstructEmbeddings constructor instead.
  embeddings = HuggingFaceInstructEmbeddings(query_instruction="Represent the query for retrieval: ")


In [38]:
instructor_embaddings = HuggingFaceInstructEmbeddings()

/var/folders/ls/6h78t28x6c35lwkn1jh_nxc80000gn/T/ipykernel_30205/355396662.py:1: LangChainDeprecationWarning: Default values for HuggingFaceInstructEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceInstructEmbeddings constructor instead.
  instructor_embaddings = HuggingFaceInstructEmbeddings()


In [39]:
instructor_embaddings

HuggingFaceInstructEmbeddings(client=INSTRUCTOR(
  (0): Transformer({'max_seq_length': None, 'do_lower_case': False, 'architecture': 'T5EncoderModel'})
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': False})
), model_name='hkunlp/instructor-large', cache_folder=None, model_kwargs={}, encode_kwargs={}, embed_instruction='Represent the document for retrieval: ', query_instruction='Represent the question for retrieving supporting documents: ', show_progress=False)

In [43]:
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents=data, embedding=instructor_embaddings)

In [44]:
vectordb

In [46]:
# When ever you have q, we will pull the same 
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInstructEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x35f4e8250>, search_kwargs={})

In [50]:
read_answer = retriever.get_relevant_documents("How can I minimize my stress from my body?")
read_answer

[Document(id='5198c108-e60c-4874-9957-474fb3fe5350', metadata={'source': 'How can I reduce my stress levels?', 'row': 6}, page_content='prompt: How can I reduce my stress levels?\nresponse: Practice mindfulness or meditation, get regular physical exercise, ensure you get enough sleep, talk to a friend or therapist, and learn to say no to excessive commitments.'),
 Document(id='dae2d346-0547-430d-98ad-1b9c2d8e54f9', metadata={'source': 'How do I create a strong password?', 'row': 4}, page_content='prompt: How do I create a strong password?\nresponse: Use a long passphrase (a series of random words), include numbers and symbols, avoid personal information, and use a unique password for each important account. A password manager can help.'),
 Document(id='09b1db49-2812-42c7-a17e-04969bf03469', metadata={'source': 'What should I do in a job interview?', 'row': 3}, page_content='prompt: What should I do in a job interview?\nresponse: Research the company thoroughly, prepare specific example

In [51]:
from langchain.chains import RetrievalQA

In [52]:
chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    input_key="query",
    return_source_documents=True
)

In [53]:
chain

RetrievalQA(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="Use the following pieces of context to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x10cf4f460>, default_metadata=(), model_kwa

In [54]:
chain("what kind of phishing emails look like ?")

/var/folders/ls/6h78t28x6c35lwkn1jh_nxc80000gn/T/ipykernel_30205/4168187109.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("what kind of phishing emails look like ?")


{'query': 'what kind of phishing emails look like ?',
 'result': 'Look for generic greetings, urgent or threatening language, spoofed sender addresses, requests for personal information, and suspicious links or attachments. When in doubt, contact the company directly.',
 'source_documents': [Document(id='c3ad54ec-2349-4006-bbb0-4cddfef467df', metadata={'source': 'What are the signs of a phishing email?', 'row': 9}, page_content='prompt: What are the signs of a phishing email?\nresponse: Look for generic greetings, urgent or threatening language, spoofed sender addresses, requests for personal information, and suspicious links or attachments. When in doubt, contact the company directly.'),
  Document(id='dae2d346-0547-430d-98ad-1b9c2d8e54f9', metadata={'source': 'How do I create a strong password?', 'row': 4}, page_content='prompt: How do I create a strong password?\nresponse: Use a long passphrase (a series of random words), include numbers and symbols, avoid personal information, and 

### This prompt template is to stop the hilousanation of the model

In [55]:
from langchain.prompts import PromptTemplate

In [63]:
prompt_template = """
    Given the following context and a question, generate an answer based on this context only.
    In the answer try to provide as much text as possible from "response" section in the source documnet context without making up answers.
    If the answer is not found in the context, kindly state "I don't know". Don't try to make up an answer.
    CONTEXT: {context}
    QUESTION: {question}
"""
PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [64]:
chain2 = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key="query",
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [65]:
chain2("Do you have JS course ?")

{'query': 'Do you have JS course ?',
 'result': "I don't know",
 'source_documents': [Document(id='09b1db49-2812-42c7-a17e-04969bf03469', metadata={'source': 'What should I do in a job interview?', 'row': 3}, page_content='prompt: What should I do in a job interview?\nresponse: Research the company thoroughly, prepare specific examples of your accomplishments using the STAR method (Situation, Task, Action, Result), ask insightful questions, and send a thank-you note afterward.'),
  Document(id='35fd60f1-4a33-4064-a5b6-2c1c192d6ffb', metadata={'source': "What is 'networking' and how do I do it effectively?", 'row': 10}, page_content="prompt: What is 'networking' and how do I do it effectively?\nresponse: Networking is building professional relationships. It's about offering help to others, not just asking for favors. Be genuine, listen more than you talk, and connect with people through events, LinkedIn, and informational interviews."),
  Document(id='5198c108-e60c-4874-9957-474fb3fe535